In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/LoL dataset/train_1226.csv')
train_target = pd.read_csv('/content/drive/MyDrive/LoL dataset/train_target_1226.csv')

test_data = pd.read_csv('/content/drive/MyDrive/LoL dataset/test_1226.csv')
test_target = pd.read_csv('/content/drive/MyDrive/LoL dataset/test_target_1226.csv')

In [ ]:
cols = train_data.select_dtypes("object").columns.tolist()

In [ ]:
cols

['league',
 'date',
 'side',
 'position',
 'playerid',
 'teamid',
 'ban1',
 'ban2',
 'ban3',
 'ban4',
 'ban5',
 'pick1',
 'pick2',
 'pick3',
 'pick4',
 'pick5']

In [ ]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
import category_encoders as ce
enc = ce.OrdinalEncoder()

tmp_train = enc.fit_transform(train_data[cols])
tmp_train.columns = [f"{col}_encoded" for col in tmp_train.columns]
train_ft = pd.concat([train_data, tmp_train], axis=1)
train_ft = train_ft.drop(columns=cols)

tmp_test = enc.transform(test_data[cols])
tmp_test.columns = [f"{col}_encoded" for col in tmp_test.columns]
test_ft = pd.concat([test_data, tmp_test], axis=1)
test_ft = test_ft.drop(columns=cols)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(


In [ ]:
import category_encoders as ce
enc = ce.OneHotEncoder()

tmp_train = enc.fit_transform(train_data[cols])
tmp_train.columns = [f"{col}_encoded" for col in tmp_train.columns]
train_ft = pd.concat([train_data, tmp_train], axis=1)
train_ft = train_ft.drop(columns=cols)


tmp_test = enc.transform(test_data[cols])
tmp_test.columns = [f"{col}_encoded" for col in tmp_test.columns]
test_ft = pd.concat([test_data, tmp_test], axis=1)
test_ft = test_ft.drop(columns=cols)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:411: FutureWarning: The `_get_tags` method is deprecated in 1.6 and will be removed in 1.7. Please implement the `__sklearn_tags__` method.
  warnings.warn(


In [ ]:
train_data.isnull().sum().sum()

1201759

In [ ]:
train_ft.columns

Index(['year', 'patch', 'league_encoded', 'date_encoded', 'side_encoded',
       'position_encoded', 'playerid_encoded', 'teamid_encoded',
       'ban1_encoded', 'ban2_encoded', 'ban3_encoded', 'ban4_encoded',
       'ban5_encoded', 'pick1_encoded', 'pick2_encoded', 'pick3_encoded',
       'pick4_encoded', 'pick5_encoded'],
      dtype='object')

In [ ]:
cols = train_ft.select_dtypes("object").columns.tolist()

In [ ]:
cols

[]

In [ ]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# 파라미터 설정
params = {
    "random_state": 42,
    "learning_rate": 0.1,  # 학습률
    "n_estimators": 500,   # 트리 개수
    "max_depth": 5,        # 최대 깊이
    "min_child_weight": 3, # 최소 자식 가중치
    "subsample": 0.8,      # 샘플링 비율
    "colsample_bytree": 0.8, # 특성 샘플링 비율
    "gamma": 0.1,           # 최소 손실 감소
    "n_jobs": -1,
    "early_stopping_rounds": 50,
    "eval_metric": "logloss"
}

# Accuracy 점수와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for train_idx, valid_idx in cv.split(train_ft, train_target):
    # 학습 데이터와 검증 데이터 분리
    x_train, x_valid = train_ft.iloc[train_idx], train_ft.iloc[valid_idx]
    y_train, y_valid = train_target.iloc[train_idx], train_target.iloc[valid_idx]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(
        x_train, y_train,
        eval_set=[(x_valid, y_valid)],
        verbose=False
    )

    # 모델 저장
    models.append(model)

    # 예측 및 Accuracy 계산
    pred = model.predict(x_valid)
    accuracy = accuracy_score(y_valid, pred)
    scores.append(accuracy)

# Accuracy의 평균 출력
print("Mean Accuracy Score:", np.mean(scores))


Mean Accuracy Score: 0.7143662607126748


- 팀 순위

In [ ]:
team_counts = train_ft.groupby(['league_encoded', 'teamid_encoded']).size().reset_index(name='count')
team_counts['team_rank'] = team_counts.groupby('league_encoded')['count'].rank(ascending=False, method='dense')

In [ ]:
train_ft.head(50)

,year,patch,league_encoded,date_encoded,side_encoded,position_encoded,playerid_encoded,teamid_encoded,ban1_encoded,ban2_encoded,ban3_encoded,ban4_encoded,ban5_encoded,pick1_encoded,pick2_encoded,pick3_encoded,pick4_encoded,pick5_encoded,team_rank
0,2022,12.01,1,1,1,1,1,1,1,1,1,1,1,149,161,164,166,168,13.0
1,2022,12.01,1,1,1,2,2,1,1,1,1,1,1,149,161,164,166,168,13.0
2,2022,12.01,1,1,1,3,3,1,1,1,1,1,1,149,161,164,166,168,13.0
3,2022,12.01,1,1,1,4,4,1,1,1,1,1,1,149,161,164,166,168,13.0
4,2022,12.01,1,1,1,5,5,1,1,1,1,1,1,149,161,164,166,168,13.0
5,2022,12.01,1,1,2,1,6,2,2,2,2,2,2,149,161,164,166,168,8.0
6,2022,12.01,1,1,2,2,7,2,2,2,2,2,2,149,161,164,166,168,8.0
7,2022,12.01,1,1,2,3,8,2,2,2,2,2,2,149,161,164,166,168,8.0
8,2022,12.01,1,1,2,4,9,2,2,2,2,2,2,149,161,164,166,168,8.0
9,2022,12.01,1,1,2,5,10,2,2,2,2,2,2,149,161,164,166,168,8.0


In [ ]:
train_ft = pd.merge(train_ft, team_counts[['league_encoded', 'teamid_encoded', 'team_rank']],
              on=['league_encoded', 'teamid_encoded'], how='left')

In [ ]:
from prophet import Prophet
import pandas as pd
from sklearn.metrics import mean_absolute_error


n_splits = 5
fold_size = len(data) // n_splits

mae_scores = []
for i in range(n_splits):
    train_data = data.iloc[:i * fold_size]
    valid_data = data.iloc[i * fold_size:(i + 1) * fold_size]

    model = Prophet()
    model.fit(train_data)

    future = valid_data[['month','day']].copy()
    forecast = model.predict(future)

    # 실제 값과 예측 값 비교
    y_true = valid_data['month','day'].values
    y_pred = forecast['month','day'].values
    mae = mean_absolute_error(y_true, y_pred)
    mae_scores.append(mae)

# MAE의 평균 출력
print("Mean MAE Score:", np.mean(mae_scores))
